**MobileNet**

In [3]:
import cv2
import numpy as np
import pyttsx3
import time

# Cargar el modelo pre-entrenado y las clases de objetos
net = cv2.dnn.readNetFromCaffe('Modelos/MobileNetSSD_deploy.prototxt', 'Modelos/MobileNetSSD_deploy.caffemodel')
classes = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

# Iniciar la cámara
cap = cv2.VideoCapture(1)  # 0 para la cámara predeterminada

while True:
    # Leer un fotograma de la cámara
    ret, frame = cap.read()

    # Preprocesar el fotograma para la detección de objetos
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    # Realizar la detección de objetos
    net.setInput(blob)
    detections = net.forward()

    # Procesar los resultados de detección
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Dibujar el recuadro y el nombre del objeto
            label = f"{classes[idx]}: {confidence * 100:.2f}%"
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar el fotograma con los objetos detectados
    cv2.imshow('Object Detection', frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()

**Voz**

In [6]:
import cv2
import numpy as np
import pyttsx3
import time
import threading

# Inicializar el motor de síntesis de voz
engine = pyttsx3.init()

# Cargar el modelo pre-entrenado y las clases de objetos
net = cv2.dnn.readNetFromCaffe('Modelos/MobileNetSSD_deploy.prototxt', 'Modelos/MobileNetSSD_deploy.caffemodel')
classes = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

# Iniciar la cámara
cap = cv2.VideoCapture(1)  # 0 para la cámara predeterminada

# Variables para controlar el tiempo y los objetos detectados
last_announcement_time = 0
announcement_interval = 5  # Segundos entre anuncios
previous_objects = set()

def announce_objects(objects):
    announcement = ", ".join(objects)
    engine.say(announcement)
    engine.runAndWait()

while True:
    # Leer un fotograma de la cámara
    ret, frame = cap.read()
    
    if not ret:
        break

    # Preprocesar el fotograma para la detección de objetos
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    # Realizar la detección de objetos
    net.setInput(blob)
    detections = net.forward()

    current_objects = set()

    # Procesar los resultados de detección
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            idx = int(detections[0, 0, i, 1])
            label = classes[idx]
            current_objects.add(label)
            
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Dibujar el recuadro y el nombre del objeto
            label_text = f"{label}: {confidence * 100:.2f}%"
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label_text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Anunciar nuevos objetos
    new_objects = current_objects - previous_objects
    if new_objects and (time.time() - last_announcement_time > announcement_interval):
        threading.Thread(target=announce_objects, args=(new_objects,)).start()
        last_announcement_time = time.time()

    previous_objects = current_objects

    # Mostrar el fotograma con los objetos detectados
    cv2.imshow('Object Detection', frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


**YoloV5**

In [7]:
import cv2
import torch

# Cargar el modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Iniciar la cámara
cap = cv2.VideoCapture(1)  # 0 para la cámara predeterminada

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar la detección de objetos
    results = model(frame)

    # Procesar los resultados y dibujar los cuadros delimitadores y etiquetas
    for *box, conf, cls in results.xyxy[0]:
        x1, y1, x2, y2 = map(int, box)
        label = f"{model.names[int(cls)]}: {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar el fotograma con los objetos detectados
    cv2.imshow('Object Detection', frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\Franco/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-22 Python-3.10.2 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
